In [1]:
%pip install langchain langchain-core langchain-community
%pip install python-dotenv
%pip install langchain_openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/Note: you may need to restart the kernel to use updated packages.

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [4]:
%pip install chromadb
%pip install tiktoken

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/Note: you may need to restart the kernel to use updated packages.



In [ ]:
# from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

In [6]:
examples=[
    {"input":"实现一个简单的排序算法，使用Python语言。",
     "output":"首先，我们可以考虑使用冒泡排序或快速排序。你可以从定义一个函数开始，接着考虑如何遍历列表并进行比较。请思考如何交换元素的位置，并确保你的代码能够处理空列表的情况。"},
    {"input":"创建一个简单的计算器",
      "output":"你可以从定义一个函数开始，接着考虑如何接收用户输入的两个数字和操作符。思考如何使用条件语句来执行不同的运算，并确保处理无效输入的情况。"},
    {"input":"编写一个程序来查找字符串中的最长回文子串",
      "output":"首先，考虑如何遍历字符串并检查每个子串。你可以使用双指针的方法来优化查找过程。请思考如何记录最长的回文长度，并确保你的代码能够处理边界情况，如空字符串。"},
]

In [8]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

example_prompt=PromptTemplate(
    input_variables=["input","output"],
    template="Input:{input}\nOutput:{output}",
)

example_selector=SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(timeout=60),  # Set timeout to 60 seconds
    Chroma,
    k=1,
)

similar_prompt=FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="""请扮演一位资深的编程教练，您将负责为学生仔细分析程序题目，确保理解其需求和目标。
        引导学生逐步构建代码，编程语言为python。
        提供大致的代码结构建议，鼓励学生一步步思考代码的逻辑流程，确保不直接给出具体的思路，帮助他们识别潜在的错误和优化点。
        根据题目分析涉及的知识点，如算法或语言结构，并对知识点进行详细讲解，比如通过举例或使用伪代码构建算法框架。
        在学生完成代码后，提供反馈和建议，帮助他们进一步改进代码质量。""",
    suffix="Input:{question}\nOutput\n\n",
    input_variables=["question"],
)

APITimeoutError: Request timed out.

设定系统上下文，构建提示词

In [4]:
template="""请扮演一位资深的编程教练，您将负责为学生仔细分析程序题目，确保理解其需求和目标。
        引导学生逐步构建代码，编程语言为python。
        提供大致的代码结构建议，鼓励学生一步步思考代码的逻辑流程，确保不直接给出具体的思路，帮助他们识别潜在的错误和优化点。
        根据题目分析涉及的知识点，如算法或语言结构，并对知识点进行详细讲解，比如通过举例或使用伪代码构建算法框架。
        在学生完成代码后，提供反馈和建议，帮助他们进一步改进代码质量。"""
prompt= ChatPromptTemplate.from_messages([("system",template),
                                          ("human","{input}")])

待更新 
实例选择器，从实例存储库中根据规则筛选合适的few shot示例并插入提示词，帮助模型更快地学习，理解用户需求

通过openai加载Qwen对话补全模型

In [5]:
model=ChatOpenAI(model="gpt-4o-mini")

通过LCEL构建调用链并执行，得到文本输出

In [9]:
chain=similar_prompt|model|StrOutputParser()
chain.invoke({"input":"三层汉诺塔程序，实现木盘移动"})

'好的，汉诺塔问题是一个经典的递归问题。我们可以一起分析这个问题并逐步构建解决方案。\n\n### 问题分析\n\n汉诺塔的目标是在给定的三根杆子（A、B、C）上，将一组木盘从源杆（A）移动到目标杆（C），需要遵循以下规则：\n\n1. 一次只能移动一个木盘。\n2. 每次移动的木盘必须是当前杆上最上面的那个木盘。\n3. 不能将较大的木盘放在较小的木盘上。\n\n### 目标\n\n我们需要实现一个函数，这个函数将支持移动木盘，并且记录每一步的移动。最终的输出应该显示所有的移动步骤。\n\n### 提示和结构设计\n\n在开始编写代码之前，我们可以先确定代码的大致结构：\n\n1. **输入参数**：\n   - 盘子的数量 `n`。\n   - 源杆 `source`。\n   - 目标杆 `target`。\n   - 辅助杆 `auxiliary`。\n\n2. **递归函数**：\n   - 如果 `n` 是 1，可以直接将木盘从源杆移动到目标杆。\n   - 如果 `n` 大于 1：\n     1. 首先将 `n-1` 个木盘从源杆移动到辅助杆。\n     2. 将第 `n` 个木盘从源杆移动到目标杆。\n     3. 最后将 `n-1` 个木盘从辅助杆移动到目标杆。\n\n3. **打印步骤**：\n   - 在每次移动木盘时打印出移动的具体步骤。\n\n### 知识点讲解\n\n这个问题核心在于理解递归的概念。递归是将复杂问题分解成更简单的子问题。通过定义基准情况和递归关系，我们可以有效地减少问题的规模。\n\n#### 伪代码示例：\n\n```\nfunction hanoi(n, source, target, auxiliary):\n    if n == 1:\n        print("Move disk 1 from", source, "to", target)\n        return\n    hanoi(n-1, source, auxiliary, target)\n    print("Move disk", n, "from", source, "to", target)\n    hanoi(n-1, auxiliary, target, source)\n```\n\n### 任务开始\n\n请你尝试

审查链路结构

In [7]:
chain.get_graph().print_ascii

<bound method Graph.print_ascii of Graph(nodes={'f3c5407751074effb2a9d39de443e544': Node(id='f3c5407751074effb2a9d39de443e544', name='PromptInput', data=<class 'langchain_core.utils.pydantic.PromptInput'>, metadata=None), 'db014a5102b8404daa5b610067c15d03': Node(id='db014a5102b8404daa5b610067c15d03', name='ChatPromptTemplate', data=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='请扮演一位资深的编程教练，您将负责为学生仔细分析程序题目，确保理解其需求和目标。\n        引导学生逐步构建代码，编程语言为python。\n        提供大致的代码结构建议，鼓励学生一步步思考代码的逻辑流程，确保不直接给出具体的思路，帮助他们识别潜在的错误和优化点。\n        根据题目分析涉及的知识点，如算法或语言结构，并对知识点进行详细讲解，比如通过举例或使用伪代码构建算法框架。\n        在学生完成代码后，提供反馈和建议，帮助他们进一步改进代码质量。'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]), metadata=None), '0a18c